# **INIT (RUN FIRST)**

In [1]:
import pandas as pd
import numpy as np
from brainflow.board_shim import BoardShim, BrainFlowInputParams, LogLevels, BoardIds
from brainflow.data_filter import DataFilter, FilterTypes
import matplotlib
import os
import torch
import pytorch_lightning as pl
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from pytorch_lightning.loggers import WandbLogger

import cv2
from torch.utils.data import DataLoader
import wandb

matplotlib.use('Agg')
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
%matplotlib inline
scaler = StandardScaler()

In [2]:
wandb.login()
api_key = 'a45abb01f9556b57620ce77c8984452bee7a8772'
board_id = 38
# eeg_names = ['TP9', 'Fp1', 'Fp2', 'TP10', 'AUX']
eeg_names = ['Fp1', 'Fp2']
sf = 256

wandb: Currently logged in as: nickhartono. Use `wandb login --relogin` to force relogin


# **DATASET AND LOADER**

In [3]:
#INI DARI GAMBAR SPECTROGRAM
class AutoencoderSpectrogramImage(Dataset):
    def __init__(self, annotations_file, dir, transform=None):
        self.file_lists = pd.read_csv(annotations_file, header=None)
        self.dir = dir
        self.transform = transform
        # self.eeg_names = ['TP9', 'Fp1', 'Fp2', 'TP10', '_AUX']
        self.eeg_names = ['Fp1', 'Fp2']

    def __len__(self):
        return len(self.file_lists)

    def __getitem__(self, idx):
        file_path = os.path.join(self.dir, self.file_lists.iloc[idx, 0].replace('/','\\'))
        file_path = file_path[:-4] + '\\'
        
        spectrograms = []
        for i in self.eeg_names:
          temp = cv2.imread(file_path + i + '.png', cv2.IMREAD_GRAYSCALE)
          spectrograms.append(temp)
        spectrograms = np.array(spectrograms)

        if self.transform:
            spectrograms = self.transform(spectrograms)
            spectrograms = spectrograms.permute(1,2,0)
        return spectrograms

In [4]:
dataset_dir = 'D:\\Nicko\\TUGAS_AKHIR\\Dataset\\Dataset_TA\\'
spectrogram_dir = 'D:\\Nicko\\TUGAS_AKHIR\\Dataset\\Dataset_TA_img\\'
training_file = dataset_dir + 'training_dir.csv'
testing_file = dataset_dir + 'testing_dir.csv'

In [5]:
#buat image
datasetTrain = AutoencoderSpectrogramImage(
    annotations_file=training_file,
    dir=spectrogram_dir,
    transform=ToTensor()
)
datasetTest = AutoencoderSpectrogramImage(
    annotations_file=testing_file,
    dir=spectrogram_dir,
    transform=ToTensor()
)

In [6]:
test = datasetTrain[99]

In [7]:
test.shape

torch.Size([2, 180, 836])

In [8]:
# split train to train and validation
# use 20% of training data for validation
train_set_size = int(len(datasetTrain) * 0.8)
valid_set_size = len(datasetTrain) - train_set_size

# split the train set into two
seed = torch.Generator().manual_seed(42)
train_set, valid_set = torch.utils.data.random_split(datasetTrain, [train_set_size, valid_set_size], generator=seed)

# data loader
train_dataloader = DataLoader(train_set, batch_size=25, shuffle=True)
validation_dataloader = DataLoader(valid_set, batch_size=25)
test_dataloader = DataLoader(datasetTest, batch_size=25)

# **AUTOENCODER MODEL**

In [9]:
#MODEL 1, 73216 param (32 x 22 x 104)
class LitAutoEncoder(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Conv2d(5, 8, 3, stride=2, padding=1),
            torch.nn.BatchNorm2d(8),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(8, 16, 3, stride=2, padding=1),
            torch.nn.BatchNorm2d(16),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(16, 32, 3, stride=2, padding=0),
            torch.nn.ReLU(True)
        )
        self.decoder = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(32, 16, 3, stride=2, 
            padding=0, output_padding=0),
            torch.nn.ReLU(True),
            torch.nn.BatchNorm2d(16),
            torch.nn.ConvTranspose2d(16, 8, 3, stride=2, 
            padding=1, output_padding=1),
            torch.nn.BatchNorm2d(8),
            torch.nn.ReLU(True),
            torch.nn.ConvTranspose2d(8, 5, 3, stride=2, 
            padding=1, output_padding=1)
        )

    def forward(self, x):
      embedding = self.encoder(x)
      return embedding

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x = batch
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = torch.nn.functional.mse_loss(x_hat, x)
        # Logging to TensorBoard (if installed) by default
        self.log("train_loss", loss, logger=True, on_epoch=True)
        return loss

    def validation_step(self, val_batch, batch_idx):
        x = val_batch
        z = self.encoder(x)
        x_hat = self.decoder(z)
        val_loss = torch.nn.functional.mse_loss(x_hat, x)
        self.log("val_loss", val_loss, logger=True, on_epoch=True)

    def test_step(self, batch, batch_idx):
        # this is the test loop
        x = batch
        z = self.encoder(x)
        x_hat = self.decoder(z)
        test_loss = torch.nn.functional.mse_loss(x_hat, x)
        self.log("test_loss", test_loss, logger=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [10]:
encoder = torch.nn.Sequential(
            torch.nn.Conv2d(5, 8, 3, stride=2, padding=1),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(8, 16, 3, stride=2, padding=1),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(16, 32, 3, stride=2, padding=0),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(32, 32, 3, stride=1, padding=0),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(32, 32, 3, stride=2, padding=1),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(32, 32, 3, stride=1, padding=0),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(32, 64, 3, stride=1, padding=0),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(64, 64, 3, stride=1, padding=0),
            torch.nn.ReLU(True)
        )
decoder = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(32, 32, 3, stride=2,
            padding=1, output_padding=1),
            torch.nn.ReLU(True),
            torch.nn.ConvTranspose2d(32, 32, 3, stride=1,
            padding=0, output_padding=0),
            torch.nn.ReLU(True),
            torch.nn.ConvTranspose2d(32, 16, 3, stride=2,
            padding=0, output_padding=0),
            torch.nn.ReLU(True),
            # torch.nn.BatchNorm2d(16),
            torch.nn.ConvTranspose2d(16, 8, 3, stride=2,
            padding=1, output_padding=1),
            # torch.nn.BatchNorm2d(8),
            torch.nn.ReLU(True),
            torch.nn.ConvTranspose2d(8, 5, 3, stride=2,
            padding=1, output_padding=1)
        )

In [11]:
result = encoder(test)
result.shape

RuntimeError: Given groups=1, weight of size [8, 5, 3, 3], expected input[1, 2, 180, 836] to have 5 channels, but got 2 channels instead

In [ ]:
decoded = decoder(result)
decoded.shape

In [ ]:
#MODEL 2, 9024 param (32 x 6 x 47)
class LitAutoEncoder2(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Conv2d(5, 8, 3, stride=2, padding=1),
            torch.nn.BatchNorm2d(8),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(8, 16, 3, stride=2, padding=1),
            torch.nn.BatchNorm2d(16),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(16, 32, 3, stride=2, padding=0),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(32, 32, 3, stride=1, padding=0),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(32, 32, 3, stride=2, padding=1),
            torch.nn.ReLU(True),
            torch.nn.BatchNorm2d(32),
            torch.nn.Conv2d(32, 32, 3, stride=1, padding=0),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(32, 32, 3, stride=1, padding=0),
            torch.nn.ReLU(True)
        )
        self.decoder = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(32, 32, 3, stride=1, 
            padding=0, output_padding=0),
            torch.nn.ReLU(True),
            torch.nn.ConvTranspose2d(32, 32, 3, stride=1, 
            padding=0, output_padding=0),
            torch.nn.BatchNorm2d(32),
            torch.nn.ReLU(True),
            torch.nn.ConvTranspose2d(32, 32, 3, stride=2, 
            padding=1, output_padding=1),
            torch.nn.ReLU(True),
            torch.nn.ConvTranspose2d(32, 32, 3, stride=1, 
            padding=0, output_padding=0),
            torch.nn.ReLU(True),
            torch.nn.ConvTranspose2d(32, 16, 3, stride=2, 
            padding=0, output_padding=0),
            torch.nn.ReLU(True),
            torch.nn.BatchNorm2d(16),
            torch.nn.ConvTranspose2d(16, 8, 3, stride=2, 
            padding=1, output_padding=1),
            torch.nn.BatchNorm2d(8),
            torch.nn.ReLU(True),
            torch.nn.ConvTranspose2d(8, 5, 3, stride=2, 
            padding=1, output_padding=1)
        )

    def forward(self, x):
      embedding = self.encoder(x)
      return embedding

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x = batch
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = torch.nn.functional.mse_loss(x_hat, x)
        # Logging to TensorBoard (if installed) by default
        self.log("train_loss", loss, logger=True, on_epoch=True)
        return loss

    def validation_step(self, val_batch, batch_idx):
        x = val_batch
        z = self.encoder(x)
        x_hat = self.decoder(z)
        val_loss = torch.nn.functional.mse_loss(x_hat, x)
        self.log("val_loss", val_loss, logger=True, on_epoch=True)

    def test_step(self, batch, batch_idx):
        # this is the test loop
        x = batch
        z = self.encoder(x)
        x_hat = self.decoder(z)
        test_loss = torch.nn.functional.mse_loss(x_hat, x)
        self.log("test_loss", test_loss, logger=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [ ]:
#MODEL 3, 12544 param (32 x 8 x 49)
class LitAutoEncoder3(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Conv2d(5, 8, 3, stride=2, padding=1),
            torch.nn.BatchNorm2d(8),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(8, 16, 3, stride=2, padding=1),
            torch.nn.BatchNorm2d(16),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(16, 32, 3, stride=2, padding=0),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(32, 32, 3, stride=1, padding=0),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(32, 32, 3, stride=2, padding=1),
            torch.nn.ReLU(True),
            torch.nn.BatchNorm2d(32),
            torch.nn.Conv2d(32, 32, 3, stride=1, padding=0),
            torch.nn.ReLU(True),
        )
        self.decoder = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(32, 32, 3, stride=1, 
            padding=0, output_padding=0),
            torch.nn.BatchNorm2d(32),
            torch.nn.ReLU(True),
            torch.nn.ConvTranspose2d(32, 32, 3, stride=2, 
            padding=1, output_padding=1),
            torch.nn.ReLU(True),
            torch.nn.ConvTranspose2d(32, 32, 3, stride=1, 
            padding=0, output_padding=0),
            torch.nn.ReLU(True),
            torch.nn.ConvTranspose2d(32, 16, 3, stride=2, 
            padding=0, output_padding=0),
            torch.nn.ReLU(True),
            torch.nn.BatchNorm2d(16),
            torch.nn.ConvTranspose2d(16, 8, 3, stride=2, 
            padding=1, output_padding=1),
            torch.nn.BatchNorm2d(8),
            torch.nn.ReLU(True),
            torch.nn.ConvTranspose2d(8, 5, 3, stride=2, 
            padding=1, output_padding=1)
        )

    def forward(self, x):
      embedding = self.encoder(x)
      return embedding

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x = batch
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = torch.nn.functional.mse_loss(x_hat, x)
        # Logging to TensorBoard (if installed) by default
        self.log("train_loss", loss, logger=True, on_epoch=True)
        return loss

    def validation_step(self, val_batch, batch_idx):
        x = val_batch
        z = self.encoder(x)
        x_hat = self.decoder(z)
        val_loss = torch.nn.functional.mse_loss(x_hat, x)
        self.log("val_loss", val_loss, logger=True, on_epoch=True)

    def test_step(self, batch, batch_idx):
        # this is the test loop
        x = batch
        z = self.encoder(x)
        x_hat = self.decoder(z)
        test_loss = torch.nn.functional.mse_loss(x_hat, x)
        self.log("test_loss", test_loss, logger=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [ ]:
#MODEL 4, 16320 param (32 x 10 x 51)
class LitAutoEncoder4(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Conv2d(5, 8, 3, stride=2, padding=1),
            torch.nn.BatchNorm2d(8),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(8, 16, 3, stride=2, padding=1),
            torch.nn.BatchNorm2d(16),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(16, 32, 3, stride=2, padding=0),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(32, 32, 3, stride=1, padding=0),
            torch.nn.ReLU(True),
            torch.nn.BatchNorm2d(32),
            torch.nn.Conv2d(32, 32, 3, stride=2, padding=1),
            torch.nn.ReLU(True),
        )
        self.decoder = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(32, 32, 3, stride=2,
            padding=1, output_padding=1),
            torch.nn.BatchNorm2d(32),
            torch.nn.ReLU(True),
            torch.nn.ConvTranspose2d(32, 32, 3, stride=1,
            padding=0, output_padding=0),
            torch.nn.ReLU(True),
            torch.nn.ConvTranspose2d(32, 16, 3, stride=2,
            padding=0, output_padding=0),
            torch.nn.ReLU(True),
            torch.nn.BatchNorm2d(16),
            torch.nn.ConvTranspose2d(16, 8, 3, stride=2,
            padding=1, output_padding=1),
            torch.nn.BatchNorm2d(8),
            torch.nn.ReLU(True),
            torch.nn.ConvTranspose2d(8, 5, 3, stride=2,
            padding=1, output_padding=1)
        )

    def forward(self, x):
      embedding = self.encoder(x)
      return embedding

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x = batch
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = torch.nn.functional.mse_loss(x_hat, x)
        # Logging to TensorBoard (if installed) by default
        self.log("train_loss", loss, logger=True, on_epoch=True)
        return loss

    def validation_step(self, val_batch, batch_idx):
        x = val_batch
        z = self.encoder(x)
        x_hat = self.decoder(z)
        val_loss = torch.nn.functional.mse_loss(x_hat, x)
        self.log("val_loss", val_loss, logger=True, on_epoch=True)

    def test_step(self, batch, batch_idx):
        # this is the test loop
        x = batch
        z = self.encoder(x)
        x_hat = self.decoder(z)
        test_loss = torch.nn.functional.mse_loss(x_hat, x)
        self.log("test_loss", test_loss, logger=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [13]:
#MODEL 5, 32640 param (64 x 10 x 51)
class LitAutoEncoder5(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Conv2d(2, 8, 3, stride=2, padding=1),
            torch.nn.BatchNorm2d(8),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(8, 16, 3, stride=2, padding=1),
            torch.nn.BatchNorm2d(16),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(16, 32, 3, stride=2, padding=0),
            torch.nn.ReLU(True),
            torch.nn.Conv2d(32, 64, 3, stride=1, padding=0),
            torch.nn.ReLU(True),
            torch.nn.BatchNorm2d(64),
            torch.nn.Conv2d(64, 64, 3, stride=2, padding=1),
            torch.nn.Flatten(),
            torch.nn.ReLU(True),
            torch.nn.Linear(32640, 16320),
            torch.nn.ReLU(True),
            torch.nn.Linear(16320, 8160),
            torch.nn.ReLU(True),
            torch.nn.Linear(8160, 4080)
        )
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(4080, 8160),
            torch.nn.ReLU(True),
            torch.nn.Linear(8160, 16320),
            torch.nn.ReLU(True),
            torch.nn.Linear(16320, 32640),
            torch.nn.ReLU(True),
            torch.nn.Unflatten(1, torch.Size([64,10,51])),
            torch.nn.ConvTranspose2d(64, 64, 3, stride=2,
            padding=1, output_padding=1),
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU(True),
            torch.nn.ConvTranspose2d(64, 32, 3, stride=1,
            padding=0, output_padding=0),
            torch.nn.ReLU(True),
            torch.nn.ConvTranspose2d(32, 16, 3, stride=2,
            padding=0, output_padding=0),
            torch.nn.ReLU(True),
            torch.nn.BatchNorm2d(16),
            torch.nn.ConvTranspose2d(16, 8, 3, stride=2,
            padding=1, output_padding=1),
            torch.nn.BatchNorm2d(8),
            torch.nn.ReLU(True),
            torch.nn.ConvTranspose2d(8, 2, 3, stride=2,
            padding=1, output_padding=1)
        )

    def forward(self, x):
      embedding = self.encoder(x)
      return embedding

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x = batch
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = torch.nn.functional.mse_loss(x_hat, x)
        # Logging to TensorBoard (if installed) by default
        self.log("train_loss", loss, logger=True, on_epoch=True)
        return loss

    def validation_step(self, val_batch, batch_idx):
        x = val_batch
        z = self.encoder(x)
        x_hat = self.decoder(z)
        val_loss = torch.nn.functional.mse_loss(x_hat, x)
        self.log("val_loss", val_loss, logger=True, on_epoch=True)

    def test_step(self, batch, batch_idx):
        # this is the test loop
        x = batch
        z = self.encoder(x)
        x_hat = self.decoder(z)
        test_loss = torch.nn.functional.mse_loss(x_hat, x)
        self.log("test_loss", test_loss, logger=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [14]:
autoencoder = LitAutoEncoder5()
autoencoder

LitAutoEncoder5(
  (encoder): Sequential(
    (0): Conv2d(2, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (7): ReLU(inplace=True)
    (8): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (9): ReLU(inplace=True)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (12): Flatten(start_dim=1, end_dim=-1)
    (13): ReLU(inplace=True)
    (14): Linear(in_features=32640, out_features=16320, bias=True)
    (15): ReLU(inplace=True)
    (16): Linear(in_features=16320, out_features=8160, bias=True)
    

In [15]:
wandb_logger = WandbLogger(project='autoencoder', save_dir='D:\\Nicko\\TUGAS_AKHIR\\AutoEncoder\\model_5')

In [16]:
trainer = pl.Trainer(max_epochs=3000, devices=1, accelerator='gpu', log_every_n_steps=9, logger=wandb_logger)
trainer.fit(autoencoder, train_dataloader, validation_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 699 M 
1 | decoder | Sequential | 699 M 
---------------------------------------
1.4 B     Trainable params
0         Non-trainable params
1.4 B     Total params
5,594.025 Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

D:\Nicko\TUGAS_AKHIR\TA_env\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
D:\Nicko\TUGAS_AKHIR\TA_env\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.99 GiB (GPU 0; 8.00 GiB total capacity; 5.37 GiB already allocated; 644.12 MiB free; 5.46 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
trainer.test(autoencoder, test_dataloader)

In [ ]:
data_test = datasetTrain[0]

In [ ]:
data_test = data_test[None, :]

In [ ]:
data_squeezed = torch.squeeze(data_test)
plt.imshow(data_squeezed[1].numpy()*255, cmap='gray')

In [ ]:
autoencoder.eval()
result = autoencoder(data_test)
decoded_result = autoencoder.decoder(result)
decoded_result.shape

In [ ]:
decoded_squeezed = torch.squeeze(decoded_result)
plt.imshow(decoded_squeezed[1].detach().numpy(), cmap='gray')

In [ ]:
autoencoder = autoencoder.load_from_checkpoint('D:\\Nicko\\TUGAS_AKHIR\\AutoEncoder\\model_4\\epoch=2999-step=27000.ckpt')